In [1]:
import sqlite3
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [2]:
conn = sqlite3.connect('data.sqlite3')

In [3]:
faculties = pd.read_sql_query('''
select * from faculties
''', con=conn)

In [10]:
usubs = pd.read_sql_query('''
select us.user_id, us.subs_id  from users_subscriptions as us
INNER JOIN

(select DISTINCT user_id from users_faculties
WHERE faculty_id in (SELECT id from faculties)
AND faculty_id in (SELECT faculty_id FROM (SELECT faculty_id, count(user_id) FROM users_faculties GROUP BY faculty_id ORDER BY count(user_id) DESC LIMIT 7))
) as uf on us.user_id=uf.user_id

WHERE us.type='page'
AND us.subs_id not in (SELECT subs_id from trash_subs)
AND us.user_id
''', con=conn)

In [13]:
usubs.to_csv('usr_sub.csv', index=False)

In [27]:
usubs['subs_id'].nunique()

44998

In [14]:
ohe = OneHotEncoder(sparse=False, dtype=int)

In [20]:
result = pd.DataFrame()

In [21]:
for chunk in pd.read_csv('usr_sub.csv', chunksize=5000):
    print(f'Chunk started')
    chunk = chunk.reset_index(drop=True)
    fitted = ohe.fit_transform(chunk[['subs_id']])
    labeled = pd.concat([chunk[['user_id']].reindex(),
                        pd.DataFrame(fitted, columns=ohe.categories_[0])],
                       axis=1)
    result.append(labeled)
    ptable = labeled.pivot_table(index='user_id',
                                 values=[c for c in labeled.columns if c != 'faculty_id'],
                                 aggfunc='sum')
    result = pd.concat([result, ptable.reset_index()],
                       sort=True).fillna(0)
    print('Chunk finished\n')

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished

Chunk started
Chunk finished



In [23]:
result = result.astype(int)

In [29]:
result = result.pivot_table(index='user_id',
                             values=[c for c in result.columns if c != 'user_id'],
                             aggfunc='sum')

In [33]:
result = result.reset_index()

In [34]:
result

,user_id,27,28,41,47,65,272,299,337,439,...,202588677,202591514,202597077,202632157,202663824,202682637,202701885,202729930,202737287,202839873
0,17680,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,82824,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,137642,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,210164,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,266484,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,544446835,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2468,568860573,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2469,569106575,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2470,587338436,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# result.to_csv('onehot_usr_sub.csv', index=False)

In [61]:
userfac = pd.read_sql_query('''
select faculty_id, user_id from users_faculties
where faculty_id in (select id from faculties)
''', con=conn)

In [64]:
userfac[
    userfac['user_id'].astype(int).isin(result['user_id'])
]['faculty_id'].nunique()

33

In [68]:
ufac = userfac[~userfac.duplicated()
        &
        userfac['faculty_id'].isin([
    '2973', '2161062', '2231790',
    '2231275', '2231267', '2173844',
    '20422'
])]

In [70]:
ufac = ufac.astype(int)

In [58]:
# ufac = userfac[~userfac['user_id'].duplicated()]

In [71]:
res = result.merge(ufac, on='user_id')

In [73]:
res.drop(columns='user_id').to_csv('fac_sub.csv', index=False)

In [76]:
res = pd.read_csv('fac_sub.csv')

In [77]:
res_x = res.drop(columns='faculty_id')
res_y = res['faculty_id']

In [78]:
X_train, X_test, y_train, y_test = train_test_split(res_x, res_y)

In [79]:
clf = GaussianNB()

In [80]:
clf.fit(X_train, y_train)

GaussianNB()

In [81]:
clf.score(X_test, y_test)

0.38110236220472443

In [83]:
res.groupby('faculty_id')['faculty_id'].count()

faculty_id
2973       371
20422      355
2161062    353
2173844    336
2231267    368
2231275    378
2231790    378
Name: faculty_id, dtype: int64

In [89]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [86]:
y_pred = clf.predict(X_test)

In [88]:
print(confusion_matrix(y_test, y_pred))

[[40 14 16  9  3 15  5]
 [ 5 29  8  3 12 19  8]
 [ 9 16 20  8  6 17  4]
 [ 9 14 10 23  4  6 22]
 [ 3 10  9  3 40 17 16]
 [ 4 17  3  7 10 32 13]
 [ 1  2  2 15 15  4 58]]


In [90]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        2973       0.56      0.39      0.46       102
       20422       0.28      0.35      0.31        84
     2161062       0.29      0.25      0.27        80
     2173844       0.34      0.26      0.29        88
     2231267       0.44      0.41      0.43        98
     2231275       0.29      0.37      0.33        86
     2231790       0.46      0.60      0.52        97

    accuracy                           0.38       635
   macro avg       0.38      0.38      0.37       635
weighted avg       0.39      0.38      0.38       635



In [103]:
pd.DataFrame({'predicted': y_pred, 'real': y_test}).to_csv('predicted.csv', index=False)

In [94]:
import pickle

In [96]:
with open('naive.pickle', 'wb') as f:
    pickle.dump(clf, f)

In [100]:
y_test

507     2173844
1936       2973
1256    2231790
2070       2973
1175    2161062
         ...   
1758    2173844
1543    2231790
699       20422
44      2161062
948     2231275
Name: faculty_id, Length: 635, dtype: int64

In [104]:
res

,27,28,41,47,65,272,299,337,439,442,...,202591514,202597077,202632157,202663824,202682637,202701885,202729930,202737287,202839873,faculty_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2161062
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2231790
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2173844
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2161062
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2534,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20422
2535,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2173844
2536,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2973
2537,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2231275


In [3]:
import pickle

In [14]:
ohe = pickle.load(open('ohe_subs.pickle', 'rb'))

In [15]:
one_line = next(pd.read_csv('fac_sub.csv', chunksize=1))

In [17]:
ohe.transform(one_line)

NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [23]:
pd.DataFrame(columns=ohe.categories)

,27,28,41,47,65,272,299,337,439,442,...,202588677,202591514,202597077,202632157,202663824,202682637,202701885,202729930,202737287,202839873


In [24]:
with open('naive.pickle', 'rb') as f:
    naive = pickle.load(f)

In [33]:
naive.predict([one_line.drop(columns='faculty_id').loc[0]])

array([2161062], dtype=int64)

In [29]:
one_line

,27,28,41,47,65,272,299,337,439,442,...,202591514,202597077,202632157,202663824,202682637,202701885,202729930,202737287,202839873,faculty_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2161062


In [35]:
groups = one_line.loc[0]

In [39]:
','.join(groups[groups == 1].index)

'1392694,2158488,5222806,7743993,12825870,16474114,17016912,18271302,21755244,22822305,24136539,24371598,25335954,27395812,30789890,33957323,33979932,34720122,38667556,43043715,45658087,48373725,48650073,48906599,48918379,49077754,52270436,52784717,53046710,54232663,54971915,55771636,56092165,56246136,56637238,56861351,56986370,57051855,57322730,57400369,64941509,73028649,75653260,76093790,87658307,94675330,118902578'